In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    pipeline
)
from torch.utils.data import DataLoader, Dataset
import torch
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm

## Получение результатов работы модели сущностей на тестовых данных

In [ ]:
label2id = {
    'B-Food': 0, 'B-Interior': 1, 'B-Price': 2, 'B-Service': 3,
    'B-Whole': 4, 'I-Food': 5, 'I-Interior': 6, 'I-Price': 7,
    'I-Service': 8, 'I-Whole': 9, 'O': 10}

id2label = {idx: label for label, idx in label2id.items()}

def get_entities(text_id, text):
    result = classifier(text)
    to_return = []
    words = []
    entities = []
    prev_entity = ''
    positions = []

    for i, token in enumerate(result):
        entity_id = int(token['entity'].split('_')[-1])
        entity = id2label[entity_id]
        start_pos = token['start']
        end_pos = token['end']

        if token['word'].startswith('##'):
            words[-1] += token['word'][2:]
            positions[-1][1] = end_pos

        elif entity.startswith('I') and entities:
            words[-1] += f' {token["word"]}'
            positions[-1][1] = end_pos

        else:
            words.append(token['word'])
            entities.append(entity)
            positions.append([start_pos, end_pos])

        prev_entity = entity

    for word, entity, position in zip(words, entities, positions):
        if entity != 'O':
            to_return.append([str(text_id), entity[2:], word, str(position[0]), str(position[1]), 'neutral'])

    return to_return

In [ ]:
classifier = pipeline("ner", model="/content/drive/MyDrive/NLP Course/bert_wo_crf_deepvk_uncased_1e02")

### Чтение тестовых данных и получение предсказаний

In [ ]:
dev_reviews = "/content/drive/MyDrive/NLP Course/Data/test/test_reviews.txt"
data = pd.read_csv(dev_reviews, sep='\t', names=['idx', 'text'])

In [ ]:
from tqdm import tqdm
all_results = []
for i, row in tqdm(data.iterrows()):
    all_results.extend(get_entities(row['idx'], row['text']))


In [ ]:
to_save = '\n'.join(['\t'.join(result) for result in all_results])

In [ ]:
with open('/content/drive/MyDrive/NLP Course/dev_pred_aspects.txt', 'w') as f:
    f.write(to_save)

### Предсказание тональности аспектов

In [ ]:
# Функция для извлечения контекста вокруг аспекта с учетом целых слов
def extract_context_with_category(text, start, end, category, window=50):
    # Находим начало контекста и пробел назад от начальной позиции аспекта
    start_idx = max(0, start - window)
    while start_idx > 0 and text[start_idx] != ' ':
        start_idx -= 1

    # Находим конец контекста и пробел вперед от конечной позиции аспекта
    end_idx = min(len(text), end + window)
    while end_idx < len(text) and text[end_idx] != ' ':
        end_idx += 1

    context = text[start_idx:end_idx].strip()
    return category + " " + context  # Добавление категории аспекта

In [ ]:
label_dict = {'negative': 0, 'neutral': 1, 'positive': 2, 'both': 3}
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "/content/drive/MyDrive/NLP Course/bert_model.pth"
model_name = 'DeepPavlov/rubert-base-cased'

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_dict))

# Загрузка сохраненных весов
model.load_state_dict(torch.load(model_path, map_location=device))

model.to(device)

In [ ]:
dev_aspects = pd.read_csv('/content/drive/MyDrive/NLP Course/dev_pred_aspects.txt', sep='\t', names=['review_id', 'aspect_category', 'aspect_text', 'start_pos', 'end_pos', 'sentiment'])
dev_reviews = pd.read_csv('/content/drive/MyDrive/NLP Course/Data/test/test_reviews.txt', sep='\t', names=['review_id', 'review_text'])

# Объединение данных
merged_data = pd.merge(dev_aspects, dev_reviews, on='review_id')

# Добавление контекста к данным
merged_data['context'] = merged_data.apply(lambda row: extract_context_with_category(row['review_text'], row['start_pos'], row['end_pos'], row['aspect_category']), axis=1)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name, model_max_length=512)

def predict_sentiment(text):
    encoded_input = tokenizer.encode_plus(
        text,
        max_length=64,
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    # Возвращаем тональность аспекта
    return list(label_dict.keys())[list(label_dict.values()).index(prediction)]

# Применение модели к каждому контексту и обновление значения тональности
dev_aspects['sentiment'] = merged_data['context'].apply(predict_sentiment)

dev_aspects.to_csv('dev_pred_aspects.txt', sep='\t', index=False)

In [ ]:
dev_aspects.to_csv('/content/drive/MyDrive/NLP Course/dev_pred_aspects.txt', sep='\t', index=False, header=False)

## Оценивание

In [ ]:
gold_test_path = "/content/drive/MyDrive/NLP Course/dev_aspects.txt"
pred_test_path = "/content/drive/MyDrive/NLP Course/dev_pred_aspects.txt"

In [ ]:
from collections import defaultdict
gold_aspect_cats = {}
with open(gold_test_path) as fg:
    for line in fg:
        line = line.rstrip('\r\n').split('\t')
        if line[0] not in gold_aspect_cats:
            gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
        gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
        gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
        gold_aspect_cats[line[0]]["cats"].append(line[1])
        gold_aspect_cats[line[0]]["sents"].append(line[5])

In [ ]:
full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
total = 0
fully_matched_pairs = []
partially_matched_pairs = []
with open(pred_test_path) as fp:
    for line in fp:
        total += 1
        line = line.rstrip('\r\n').split('\t')
        start, end = int(line[3]), int(line[4])
        category = line[1]
        doc_gold_aspect_cats = gold_aspect_cats[line[0]]
        if start in doc_gold_aspect_cats["starts"]:
            i = doc_gold_aspect_cats["starts"].index(start)
            if doc_gold_aspect_cats["ends"][i] == end:
                full_match += 1
                if doc_gold_aspect_cats["cats"][i] == category:
                    full_cat_match += 1
                else:
                    partial_cat_match += 1
                fully_matched_pairs.append(
                    (
                        [
                            doc_gold_aspect_cats["starts"][i],
                            doc_gold_aspect_cats["ends"][i],
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                        ],
                        line
                    )
                )
                continue
        for s_pos in doc_gold_aspect_cats["starts"]:
            if start <= s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if doc_gold_aspect_cats["ends"][i] == end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i],
                                doc_gold_aspect_cats["ends"][i],
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    continue
                matched = False
                for e_pos in doc_gold_aspect_cats["ends"][i:]:
                    if s_pos <= end <= e_pos:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i],
                                    doc_gold_aspect_cats["ends"][i],
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        matched = True
                        break
                if matched:
                    break
            if start > s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if start < doc_gold_aspect_cats["ends"][i] <= end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i],
                                doc_gold_aspect_cats["ends"][i],
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    break

### Метрики по сущностям

In [ ]:
gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")

### Метрики по тональностям аспектов

In [ ]:
def sentiment_accuracy(matches):
    matched_sentiment = 0.
    for pair in matches:
        *_, gold_s = pair[0]
        *_, pred_s = pair[1]
        if gold_s == pred_s:
            matched_sentiment += 1
    print(f"Mention sentiment accuracy: {matched_sentiment / len(matches)}")

In [ ]:
sentiment_accuracy(fully_matched_pairs)

In [ ]:
sentiment_accuracy(partially_matched_pairs)